# percentil 80 20   SignalStatistics2

In [1]:
import pandas as pd
import dask.dataframe as dd
# loading metadata file
filePath = '/'
df = dd.read_csv(filePath + '.csv').drop_duplicates()

# Pre-processing to filter out inconsistent data
df = df[df['Number of Remote Sensors'] < df['Number of Remote Sensors'].quantile(.999)]
df = df[df['Number of Occupants'] < df['Number of Occupants'].quantile(.999)]
df = df[df['Number of Floors'] < df['Number of Floors'].quantile(.999)]
df = df[df['Floor Area [ft2]'] < df['Floor Area [ft2]'].quantile(.999)]

df = df[(df['Country']=='US') | (df['Country']=='CA')]

"df = df[df['Number of Occupants'] < df['Number of Occupants'].quantile(.999)]\ndf = df[df['Number of Floors'] < df['Number of Floors'].quantile(.999)]\ndf = df[df['Floor Area [ft2]'] < df['Floor Area [ft2]'].quantile(.999)]\n\ndf = df[(df['Country']=='US') | (df['Country']=='CA')]"

In [2]:
from pathlib import Path
lista = []
# List all files in directory using pathlib
path1=1/"
basepath = Path(path1)
files_in_basepath = (entry for entry in basepath.iterdir() if entry.is_file())
for item in files_in_basepath:
    lista.append(item.name)
path2="2/"
basepath = Path(path2)
files_in_basepath = (entry for entry in basepath.iterdir() if entry.is_file())
for item in files_in_basepath:
    lista.append(item.name)


In [4]:
lista_FileName = [s.replace('.csv', '') for s in lista]  # to have only the name of the file

In [5]:
# Create dataset with only Identifiers in 2019 
df_in_2019 = df[df.Identifier.isin(lista_FileName)]

In [6]:
# Creation of datasets for 1 and 2 occupants
df1 = df_in_2019[(df_in_2019['Number of Occupants'] == 1 ) & (df_in_2019['Number of Remote Sensors'] >= 3)]
df2 = df_in_2019[(df_in_2019['Number of Occupants'] == 2 ) & (df_in_2019['Number of Remote Sensors'] >= 3)]

In [7]:
# Merge to have identifiers that are in the percentile

In [8]:
df_percentilID1 = pd.read_csv('.csv')
df_percentilID2 = pd.read_csv('.csv')

In [10]:
# valid data
df1=dd.merge(df1, df_percentilID1['0'], left_on='Identifier',right_on='0')
df2=dd.merge(df2, df_percentilID2['0'], left_on='Identifier',right_on='0')

# excluded from valid data
#df1=df1[(~df1.Identifier.isin(df_percentilID1['0']))]
#df2=df2[(~df2.Identifier.isin(df_percentilID2['0']))]

In [11]:
df2 = df2.sample(frac=(len(df1)/len(df2)), random_state=0)

In [12]:
#df2 = df2[(~df2.Identifier.isin(df2_aa.Identifier.compute()))]

In [16]:
# Take all Identifiers of df2, to after open them
a=df2.loc[:,'Identifier'].compute().tolist()
lista=[]
for i in a:
    lista.append(mainPath+i+'.csv')

In [17]:
len(lista)

642

In [18]:
# Read all files in the df2['Identifier'] column
ff=dd.read_csv(lista,  parse_dates=['DateTime'], include_path_column=True).drop('Unnamed: 0',axis=1)
ff['path']=ff['path'].str.replace(path2, '')
ff['path']=ff['path'].str.replace('.csv', '')

In [19]:
# Manage of columns

ff['dayofweek']=ff['DateTime'].dt.day_name()
ff['Time'] = ff['DateTime'].dt.time
ff['Month'] = ff['DateTime'].dt.month

In [20]:
ff_columns = ['path', 'mean_activation', 'dayofweek', 'Time', 'Month']
result = dd.merge(df2[['Identifier','Number of Remote Sensors']], ff[ff_columns], left_on='Identifier', right_on='path').drop(['path'],axis=1)
#result['mean_activation']=result['activation']/(result['working_sensors']) 

In [21]:
aa=result[['Identifier','dayofweek','Time','mean_activation']].groupby(['Identifier','Time','dayofweek']).mean()
pp=dd.from_array(aa.to_records()) 

In [22]:
pp["TimeSTR"]=pp["Time"].apply((lambda x: x.strftime('%H:%M')), meta='str')
aa2 = pp.copy()
new = aa2["TimeSTR"]
  
# concatenating team with name column 
# overwriting name column 
aa2["dayofweek"]= aa2["dayofweek"].str.cat(new, sep =", ") 

In [23]:

bb=aa2.drop(['Time'],axis=1)
bb = bb.categorize(columns='dayofweek')
table = dd.pivot_table(bb, values='mean_activation', index='Identifier',
                    columns='dayofweek')

In [24]:
t=dd.from_array(table.to_records())
result_f = dd.merge(df2,t, on='Identifier').drop(['filename'],axis=1)

In [25]:
(result_f.compute()).to_csv('.csv')


In [ ]:
# Trying seaborn std plot

In [4]:
import pandas as pd
import seaborn as sns
filePath = '/'
df_1ocupante = pd.read_csv(filePath + '.csv', index_col=0)
df_2ocupantes = pd.read_csv(filePath + '.csv', index_col=0)

In [5]:
seed=2020
df_pp = pd.concat([df_1ocupante, df_2ocupantes])
df_pp = df_pp.reset_index()

# Drop data with missing values
df_pp = df_pp.loc[df_pp.iloc[:, 22:].dropna().index]
print(df_pp.shape)
# Resample dataframe
resample_value = df_pp['Number of Occupants'].value_counts().min()
df_pp = pd.concat([
    df_pp[df_pp['Number of Occupants'] == 1].sample(resample_value,random_state=seed),
    df_pp[df_pp['Number of Occupants'] == 2].sample(resample_value,random_state=seed)
])
df_pp = df_pp.sample(frac=1,random_state=seed).reset_index(drop=True)

(1152, 2039)


In [6]:
df_df=df_pp.iloc[:, 22:]

In [7]:
df_df[['Number of Occupants','Identifier']] = df_pp[['Number of Occupants','Identifier']]

In [8]:
fmri=df_df.drop(['Identifier','0'],axis=1).melt(id_vars=["Number of Occupants"], 
        var_name="Date", 
        value_name="Value")

In [9]:
fmri1=fmri[fmri['Number of Occupants']==1]
fmri2=fmri[fmri['Number of Occupants']==2]

In [10]:
pp1=fmri1.drop('Number of Occupants',axis=1).groupby(['Date']).describe(percentiles=[0.05, 0.95, 0.10,0.90,0.15,0.85,0.20,0.80])
pp2=fmri2.drop('Number of Occupants',axis=1).groupby(['Date']).describe(percentiles=[0.05, 0.95, 0.10,0.90,0.15,0.85,0.20,0.80])

In [11]:
palette = sns.color_palette("husl", 1)
colors = sns.color_palette("husl", 2)
#sns.lineplot(ax=ax,data=fmri1, x="Date", y="Value", hue="Number of Occupants",ci=95,palette=palette)

In [12]:
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
fig, ax = plt.subplots(figsize=(30,8))
xtick= np.arange(start=0, stop=len(pp1.index), step=72)
plt.xticks(rotation=90)
plt.setp(ax, xticks=xtick, xticklabels=pp1.index[xtick])

ax.set_title( 'Filter percentile 80-20, average activation of each day of the week every 5 minutes throughout the year')

sns.lineplot(ax=ax,data=pp1['Value'][['mean']],palette=palette,legend=False)

sns.lineplot(ax=ax,data=pp2['Value'][['mean']],legend=False)
ax.legend(['One occupant','Two occupants'])

In [ ]:
fig, ax = plt.subplots(figsize=(30,8))

sns.lineplot(ax=ax,data=pp1['Value'][['mean']],palette=palette)
ax.fill_between(pp1.index, y1=pp1['Value'][['5%']]['5%'], y2=pp1['Value'][['95%']]['95%'],alpha=0.2,facecolor=palette)#,alpha=0.40,  facecolor = colors)
sns.lineplot(ax=ax,data=pp2['Value'][['mean']])
ax.fill_between(pp2.index, y1=pp2['Value'][['5%']]['5%'], y2=pp2['Value'][['95%']]['95%'],alpha=0.2)